In [8]:

import time
import pandas as pd
import re
import numpy as np
from konlpy.tag import Kkma, Okt
from sklearn.model_selection import train_test_split
import kss
import time
import os
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sn
import matplotlib.font_manager as fm
%matplotlib inline

In [9]:
import os
import re
from keras import regularizers
import keras
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from keras import backend as K
from keras.models import Model
from keras import initializers
from keras.engine.topology import Layer
from keras.layers import Dense, Input, RepeatVector, Permute, Multiply, Lambda,BatchNormalization
from keras.layers import Embedding, GRU, Bidirectional, TimeDistributed, LSTM, Dropout
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.utils.np_utils import to_categorical
from nltk import tokenize
import tensorflow as tf
from gensim.models import Word2Vec
from keras.preprocessing.sequence import pad_sequences

import sentencepiece as spm
from keras.callbacks import EarlyStopping


In [10]:
class Attention(Layer):
    def __init__(self, regularizer = None, **kwargs):
        super(Attention, self).__init__(**kwargs)
        self.regularizer = regularizer
        self.supports_masking = True

    def build(self, input_shape):
        self.context = self.add_weight(name='context', shape=(input_shape[-1], 1),initializer=initializers.RandomNormal(mean=0.0, stddev=0.05, seed=None),regularizer=self.regularizer,trainable=True)
        super(Attention, self).build(input_shape)
    def compute_mask(self, inputs, mask=None):
        return None

    def call(self, x, mask=None):
        attention_in = K.exp(K.squeeze(K.dot(x, self.context), axis=-1))
        attention = attention_in/K.expand_dims(K.sum(attention_in, axis=-1), -1)

        if mask is not None:
        # use only the inputs specified by the mask
        # import pdb; pdb.set_trace()
            attention = attention*K.cast(mask, 'float32')

        weighted_sum = K.batch_dot(K.permute_dimensions(x, [0, 2, 1]), attention)
        return weighted_sum

    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[-1]


In [11]:
! nvidia-smi

Wed Jul  8 09:44:06 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 396.37                 Driver Version: 396.37                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:3B:00.0 Off |                    0 |
| N/A   32C    P0    29W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla P100-PCIE...  Off  | 00000000:D8:00.0 Off |                    0 |
| N/A   27C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
        

In [12]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [13]:
yahoo_data = pd.read_csv('train.csv')

In [14]:
yahoo_data.columns = ['category', 'title', 'content', 'answer']
yahoo_data

,category,title,content,answer
0,6,What is the best off-road motorcycle trail ?,long-distance trail throughout CA,i hear that the mojave road is amazing!<br />\...
1,3,What is Trans Fat? How to reduce that?,I heard that tras fat is bad for the body. Wh...,Trans fats occur in manufactured foods during ...
2,7,How many planes Fedex has?,I heard that it is the largest airline in the ...,according to the www.fedex.com web site:\nAir ...
3,7,"In the san francisco bay area, does it make se...",the prices of rent and the price of buying doe...,renting vs buying depends on your goals. <br /...
4,5,What's the best way to clean a keyboard?,I have very small stuff stuck under my keyboar...,"There are commercial kits available, but a can..."
5,2,Why do people blush when they are embarrassed?,Why do people blush when they are embarrassed?,from ask yahoo...\nhttp://ask.yahoo.com/ask/20...
6,8,"Is Lin Qingxia (aka Brigitte Lin) ""the most be...",This is according to Stephen Chow (http://www....,Well. Everyone has different definition on wh...
7,5,"What is the origin of ""foobar""?",I want to know the meaning of the word and how...,"Not sure if this is the origin, but I think it..."
8,2,How the human species evolved?,How the human species evolved?,A tough question as it overlaps science and th...
9,4,Who said the statement below and what does it ...,Can someone help me with understanding someone...,That is kind of a tricky little quote. Sometim...


In [15]:
def title_content_combine(title,content, answer):
    return str(title) + ' ' + str(content) + ' ' + str(answer)

In [16]:
yahoo_data['x'] = yahoo_data.apply(lambda x : title_content_combine(x.title, x.content, x.answer), axis = 1)

In [17]:
data = yahoo_data[['x', 'category']]

In [18]:
max_nb_instances = 300000
x  = pd.DataFrame(data.x)
y = pd.DataFrame(data.category)
y.columns = ['category']
x_train = x[:max_nb_instances]
y_train = y[:max_nb_instances]
y_train.category = y_train.category - 1

/home/bigdata/.conda/envs/py36/lib/python3.6/site-packages/pandas/core/generic.py:4405: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [19]:
from keras.preprocessing.text import Tokenizer
MAX_NB_WORDS = 300000

In [20]:
contents = []
texts = []
for text in x_train.x:
    sentences = tokenize.sent_tokenize(text)
    for sen in sentences:
        texts.append(sen)
    contents.append(sentences)

In [21]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)

In [22]:
MAX_SENTENCES = 15
MAX_SENTENCE_LENGTH = 100
X_train = np.zeros((max_nb_instances, MAX_SENTENCES, MAX_SENTENCE_LENGTH), dtype='int32')
for i, sentences in enumerate(contents):
    for j, sent in enumerate(sentences):
        if j < MAX_SENTENCES:
            wordTokens = text_to_word_sequence(sent)
            k = 0
            for _, word in enumerate(wordTokens):
                if k < MAX_SENTENCE_LENGTH:
                    X_train[i, j, k] = tokenizer.word_index[word]
                    k = k + 1

In [23]:
word_index = tokenizer.word_index
print('Total %s unique tokens.' % len(word_index))

Total 365973 unique tokens.


In [24]:
y_train.columns = ['category']
Y_train = to_categorical(np.asarray(y_train))

In [25]:
validation_split = 0.1
nb_validation_samples = int(validation_split * X_train.shape[0])

In [26]:
x_train = X_train[:-nb_validation_samples]
y_train = Y_train[:-nb_validation_samples]
x_val = X_train[-nb_validation_samples:]
y_val = Y_train[-nb_validation_samples:]

In [27]:
embeddings_index = {}
f = open('glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [28]:
print('Total %s word vectors.' % len(embeddings_index))

Total 400000 word vectors.


In [29]:
embedding_dim = 100
embedding_matrix = np.zeros((len(word_index)+1, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [30]:
l2_reg = regularizers.l2(1e-8)

sentence_in = Input(shape=(MAX_SENTENCE_LENGTH,), dtype='int32')
embedded_word_seq = Embedding(len(word_index) +1,embedding_dim, weights=[embedding_matrix], input_length = MAX_SENTENCE_LENGTH, trainable= True, mask_zero=True, name='word_embeddings',)(sentence_in)
word_encoder = Bidirectional(GRU(50, return_sequences=True, kernel_regularizer=l2_reg))(embedded_word_seq)
dense_transform_w = Dense(100, activation='relu', name='dense_transform_w', kernel_regularizer=l2_reg)(word_encoder)
attention_weighted_sentence = Model(sentence_in, Attention(name='word_attention', regularizer=l2_reg)(dense_transform_w))
word_attention_model = attention_weighted_sentence


texts_in = Input(shape=(MAX_SENTENCES, MAX_SENTENCE_LENGTH), dtype='int32')
attention_weighted_sentences = TimeDistributed(attention_weighted_sentence)(texts_in)
sentence_encoder = Bidirectional(GRU(50, return_sequences=True, kernel_regularizer=l2_reg))(attention_weighted_sentences)
dense_transform_s = Dense(100, activation='relu', name='dense_transform_s',kernel_regularizer=l2_reg)(sentence_encoder)
attention_weighted_text = Attention(name='sentence_attention', regularizer=l2_reg)(dense_transform_s)
prediction = Dense(y_train.shape[1], activation='softmax')(attention_weighted_text)
model = Model(texts_in, prediction)

model.compile(optimizer=keras.optimizers.Adam(lr=0.001),loss='categorical_crossentropy',metrics=['acc'])


In [31]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 15, 100)           0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, 15, 100)           36652900  
_________________________________________________________________
bidirectional_2 (Bidirection (None, 15, 100)           45300     
_________________________________________________________________
dense_transform_s (Dense)    (None, 15, 100)           10100     
_________________________________________________________________
sentence_attention (Attentio (None, 100)               100       
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1010      
Total params: 36,709,410
Trainable params: 36,709,410
Non-trainable params: 0
________________________________________________________________

In [33]:
es = EarlyStopping(monitor = 'val_loss')


hist = model.fit(x=[x_train],
          y=[y_train],
          batch_size=512,
          epochs=2,
          shuffle=True,
          callbacks=[es],
          validation_data=([x_val], 
                           [y_val]))

Train on 270000 samples, validate on 30000 samples
Epoch 1/2
270000/270000 [==============================] - 257s 950us/step - loss: 1.2361 - acc: 0.5739 - val_loss: 1.0170 - val_acc: 0.6513
Epoch 2/2
270000/270000 [==============================] - 251s 928us/step - loss: 0.9018 - acc: 0.6900 - val_loss: 0.9752 - val_acc: 0.6654


In [34]:
def normalize(text):
    reviews = []
    texts = []
    return tokenize.sent_tokenize(text) 

def encode_input(reviews):
    
    
    
    ret = np.zeros((MAX_SENTENCES, MAX_SENTENCE_LENGTH), dtype='int32')
    for j, sent in enumerate(reviews):
        if j < MAX_SENTENCES:
            wordTokens = text_to_word_sequence(sent)
            k = 0
            for _, word in enumerate(wordTokens):
                if k< MAX_SENTENCE_LENGTH:
                    ret[j][k] = tokenizer.word_index[word]
                    k = k + 1
    return ret

In [37]:
yahoo_test = pd.read_csv('test.csv')
yahoo_test.columns = ['category', 'title', 'content', 'answer']
yahoo_test['x'] = yahoo_test.apply(lambda x : title_content_combine(x.title, x.content, x.answer), axis = 1)
test_data = yahoo_test[['x', 'category']]
test_data.category =  test_data.category -1

In [52]:
import random
rnd = random.randint(0, test_data.shape[0])
rnd

32231

In [53]:
normalized_text = normalize(test_data.x[rnd])
encoded_text = encode_input(normalized_text)

hidden_word_encoding_out = Model(inputs=word_attention_model.input,outputs=word_attention_model.get_layer('dense_transform_w').output)
hidden_word_encodings = hidden_word_encoding_out.predict(encoded_text)
word_context = word_attention_model.get_layer('word_attention').get_weights()[0]
u_wattention = encoded_text*np.exp(np.squeeze(np.dot(hidden_word_encodings, word_context)))


reverse_word_index = {value:key for key,value in tokenizer.word_index.items()}



nopad_encoded_text = encoded_text[-len(text):]
nopad_encoded_text = [list(filter(lambda x: x > 0, sentence)) for sentence in nopad_encoded_text]
reconstructed_texts = [[reverse_word_index[int(i)] for i in sentence] for sentence in nopad_encoded_text]
nopad_wattention = u_wattention[:len(normalized_text)]
nopad_wattention = nopad_wattention/np.expand_dims(np.sum(nopad_wattention, -1), -1)
nopad_wattention = np.array([attention_seq[:len(sentence)] for attention_seq, sentence in zip(nopad_wattention, nopad_encoded_text)])
word_activation_maps = []


for i, att in enumerate(nopad_wattention):
    word_activation_maps.append(list(zip(reconstructed_texts[i], att)))

hidden_sentence_encoding_out = Model(inputs=model.input, outputs=model.get_layer('dense_transform_s').output)
hidden_sentence_encodings = np.squeeze(hidden_sentence_encoding_out.predict(np.expand_dims(encoded_text, 0)), 0)
sentence_context = model.get_layer('sentence_attention').get_weights()[0]
u_sattention = np.exp(np.squeeze(np.dot(hidden_sentence_encodings, sentence_context), -1))
nopad_sattention = u_sattention[-len(normalized_text):]

nopad_sattention = nopad_sattention/np.expand_dims(np.sum(nopad_sattention, -1), -1)

activation_map = list(zip(word_activation_maps, nopad_sattention))


In [54]:
activation_map

[([('i', 0.00041692316530951513),
   ('have', 0.0005191419598680311),
   ('swollen', 0.12817079383049274),
   ('lymph', 0.3665149002827043),
   ('nodes', 0.3835289149950114),
   ('under', 0.005693741535427932),
   ('my', 0.0002219778329290879),
   ('arm', 0.05090700447124568),
   ('how', 0.0002743539796024822),
   ('can', 7.640418437593212e-05),
   ('i', 2.688268294229652e-05),
   ('ease', 0.061973419428466264),
   ('the', 4.324538366192062e-06),
   ('pain', 0.0016712171132581381)],
  0.1842726),
 ([('i', 5.2202043241161986e-05),
   ('just', 0.00013099881652882438),
   ('started', 0.002222066829953164),
   ('taking', 0.002114863182943874),
   ('antibotics', 0.9740055772808796),
   ('today', 0.0025672141239150735),
   ('but', 3.637832692533505e-05),
   ('under', 0.0006561657686550541),
   ('my', 2.687123145598433e-05),
   ('arm', 0.009710255273680964),
   ('is', 5.260919616452161e-06),
   ('so', 1.7216611301052583e-05),
   ('swollen', 0.007542660723833892),
   ('the', 6.366783050044455e

In [55]:
ans = model.predict(np.reshape(encoded_text, (1,)+encoded_text.shape))

In [58]:
ans.argsort()[0]

array([9, 7, 4, 0, 8, 5, 3, 1, 6, 2])

In [59]:
test_data.category[rnd]

2